In [1]:
from keras.datasets import imdb
from keras.preprocessing import sequence
from keras.engine.topology import Layer
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout, Flatten, TimeDistributed, LSTM, Embedding
from keras.utils import to_categorical
from keras import optimizers

import t3f
import numpy as np

/usr/local/lib/python3.5/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
max_features = 20000
maxlen = 80  # cut texts after this number of words (among top max_features most common words)
batch_size = 32

print('Loading data...')
(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=max_features)
print(len(x_train), 'train sequences')
print(len(x_test), 'test sequences')

print('Pad sequences (samples x time)')
x_train = sequence.pad_sequences(x_train, maxlen=maxlen)
x_test = sequence.pad_sequences(x_test, maxlen=maxlen)
print('x_train shape:', x_train.shape)
print('x_test shape:', x_test.shape)

Loading data...
25000 train sequences
25000 test sequences
Pad sequences (samples x time)
x_train shape: (25000, 80)
x_test shape: (25000, 80)


## LSTM model

In [6]:
print('Build lstm model...')
model = Sequential()
model.add(Embedding(max_features, 128))
model.add(LSTM(128, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(1, activation='sigmoid'))

# try using different optimizers and different optimizer configs
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

Build lstm model...


In [7]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, None, 128)         2560000   
_________________________________________________________________
lstm_2 (LSTM)                (None, 128)               131584    
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 129       
Total params: 2,691,713
Trainable params: 2,691,713
Non-trainable params: 0
_________________________________________________________________


In [8]:
print('Train...')
model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=3,
          validation_data=(x_test, y_test))
score, acc = model.evaluate(x_test, y_test,
                            batch_size=batch_size)
print('Test score:', score)
print('Test accuracy:', acc)

Train...
Train on 25000 samples, validate on 25000 samples
Epoch 1/3
25000/25000 [==============================] - 166s - loss: 0.4613 - acc: 0.7834 - val_loss: 0.3857 - val_acc: 0.8308
Epoch 2/3
25000/25000 [==============================] - 164s - loss: 0.2968 - acc: 0.8796 - val_loss: 0.4019 - val_acc: 0.8143
Epoch 3/3
25000/25000 [==============================] - 43s    
Test score: 0.42779255952119827
Test accuracy: 0.82548


## TT-LSTM model

In [9]:
from lstm import TT_LSTM

In [10]:
print('Build lstm model...')
model = Sequential()
model.add(Embedding(max_features, 128))
model.add(TT_LSTM(row_dims=[4, 4, 2, 4], column_dims=[4, 4, 2, 4], tt_rank=4))
model.add(Dense(1, activation='sigmoid'))

# try using different optimizers and different optimizer configs
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

Build lstm model...
(?, 512)
(?, 128)
A TT-Matrix variable of size 128 x 512, underlying tensor shape: (4, 4, 2, 4) x (16, 4, 2, 4), TT-ranks: (1, 4, 4, 4, 1)
(?, 512)
(?, 512)
(?, 128)
A TT-Matrix variable of size 128 x 512, underlying tensor shape: (4, 4, 2, 4) x (16, 4, 2, 4), TT-ranks: (1, 4, 4, 4, 1)
(?, 512)


In [11]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (None, None, 128)         2560000   
_________________________________________________________________
tt_lstm_1 (TT_LSTM)          (None, 128)               1280      
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 129       
Total params: 2,561,409
Trainable params: 2,561,409
Non-trainable params: 0
_________________________________________________________________


In [12]:
print('Train...')
model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=3,
          validation_data=(x_test, y_test))
score, acc = model.evaluate(x_test, y_test,
                            batch_size=batch_size)
print('Test score:', score)
print('Test accuracy:', acc)

Train...
Train on 25000 samples, validate on 25000 samples
Epoch 1/3
25000/25000 [==============================] - 320s - loss: 0.4184 - acc: 0.8030 - val_loss: 0.3558 - val_acc: 0.8421
Epoch 2/3
25000/25000 [==============================] - 315s - loss: 0.2271 - acc: 0.9104 - val_loss: 0.3821 - val_acc: 0.8370
Epoch 3/3
25000/25000 [==============================] - 98s    
Test score: 0.5210889452075959
Test accuracy: 0.82504


Here we have same results as common lstm in same number of iterations. 